# Sesión práctica de introducción a los *generative large language models (LLMs)*

**Índice**
- [Tipos de modelos de lenguaje y ecosistema actual](#toc0_)
- [Primeros pasos con un modelo de lenguaje](#toc1_)
- [Ejemplo práctico: no generativos vs instruction tuned VS NO instruction tuned** ](#toc12_)
- [Estrategias y parámetros de generación](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
DEVICE = "cuda:0" # Si no tenéis GPU cambiar a "cpu"
# Si usáis un Google Colab ó un entorno con GPU podéis dejarlo en "cuda:0"

## Útiles de programación que usaremos a lo largo del notebook

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers.generation.utils import GenerateDecoderOnlyOutput, GenerateBeamDecoderOnlyOutput
import torch
from torch import nn


def describir_modelo(model: AutoModelForCausalLM):
    n_params = model.num_parameters()
    print(f" - El modelo tiene {n_params/1e6:,.1f} millones parámetros") # ~0.5 billones (1 billon = 1,000 millones en inglés) de parámetros
    print(f" - Arquitectura: {model.config.architectures[0]}")
    try:
        ctx = model.config.max_position_embeddings
    except:
        ctx = model.config.seq_length
    print(f" - Contexto máximo: {ctx:,} tokens")
    print(f" - Formato de los parámetros: {model.config.torch_dtype}")
    print(f" - Tamaño del vocabulario: {model.config.vocab_size:,} tokens")


def crear_prompt_chat_model(
        mensaje_usuario: str,
        tokenizer: AutoTokenizer,
        historial: str = None,
        mensaje_sistema: str = "You are a helpful assistant."
) -> str:

    if historial is None:
        messages = [{"role": "system", "content": mensaje_sistema}]
    else:
        messages = []

    messages.append({"role": "user", "content": mensaje_usuario})
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    if historial is not None:
        prompt = f"{historial} {prompt}"
    return prompt


def responde_al_mensaje(
        model: AutoModelForCausalLM,
        tokenizer: AutoTokenizer,
        prompt: str,
        streamer: TextStreamer = None,
        gen_kwargs: dict = None,
        return_scores: bool = False,
        device: str = "cuda:0"
    ) -> list[str] | tuple[list[str], GenerateDecoderOnlyOutput]:

    gen_kwargs = gen_kwargs or {}
    if return_scores:
        gen_kwargs["return_dict_in_generate"] = True
        gen_kwargs["output_scores"] = True

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    model_output = model.generate(
        input_ids,
        pad_token_id=tokenizer.eos_token_id,
        **gen_kwargs,
        streamer=streamer,
    )
    if isinstance(model_output, GenerateDecoderOnlyOutput):
        # Será el caso si en model.generate hemos pasado return_dict_in_generate=True
        response_tokens = model_output.sequences
    elif isinstance(model_output, torch.Tensor):
        response_tokens = model_output
    elif isinstance(model_output, GenerateBeamDecoderOnlyOutput):
        response_tokens = model_output.sequences
    else:
        raise ValueError(f"El modelo ha devuelto un tipo inesperado: {type(model_output)}")


    responses_txt = tokenizer.batch_decode(response_tokens[:,len(input_ids[0]):], skip_special_tokens=True)

    return responses_txt, model_output


def inspeccionar_probabilidades(
        model_output: GenerateDecoderOnlyOutput,
        response_txt: list[str],
        tokenizer: AutoTokenizer,
        top_n_probables: int = 5
) -> None:
    response_tokens = tokenizer(response_txt, return_tensors="pt").input_ids.to(DEVICE)[0]
    # Añadimos el token de EoS (End of Sentence)
    response_tokens = torch.cat([response_tokens, torch.tensor([tokenizer.eos_token_id], device=DEVICE)])

    # Probabilidades de cada token en cada elemento de la respuesta
    probs_por_paso = [nn.functional.softmax(scores, dim=-1) for scores in model_output.scores]

    # Cada elemento de la lista `probs_por_paso` es un tensor de shape (1, n_vocab),
    # y contiene la probabilidad de elegir cada uno de los tokens del vocabulario
    print(f"Probabilidades asociadas a cada token", end='')
    if top_n_probables != 0:
        print(f" - [Candidatos más probables]", end='')
    for token_elegido_paso_i, probs_paso_i in zip(response_tokens, probs_por_paso):
        prob_of_tok_i = probs_paso_i[0,token_elegido_paso_i]
        # Índices de los tokens más probables
        print(f"\n  '{tokenizer.decode(token_elegido_paso_i)}'({prob_of_tok_i:.0%})", end='')
        if top_n_probables != 0:
            print("- [", end='')
            top_n_tokens = torch.argsort(probs_paso_i[0], descending=True)[:top_n_probables]
            probs_de_top_n = probs_paso_i[0,top_n_tokens]
            # Excluir si alguna es 0
            top_n_tokens = top_n_tokens[probs_de_top_n > 0]
            probs_de_top_n = probs_de_top_n[probs_de_top_n > 0]
            for token_idx in top_n_tokens:
                token_txt = tokenizer.decode(token_idx)
                print(f"'{token_txt}'({probs_paso_i[0,token_idx]:.0%})", end='')
            print("]", end='')

    return

## <a id='toc0_'></a> Tipos de modelos de lenguaje y ecosistema actual

Como vimos en el *fastbook*, los **modelos generativos** son un subconjunto (cada vez más importante, eso sí) de los foundational models de lenguaje, y no todos los modelos generativos saben seguir intrucciones / son conversacionales!


<div style="text-align:center">
    <img src="https://lh3.googleusercontent.com/d/1ZOMiZRqe1-JgRHd5VLrFrUIA9NSvSWhh" width="600">
</div>


Hoy nos centraremos sobre todo en los modelos de lenguaje generativo, con foco en los *instruction-tuned*, que son los más importantes y con más potencial a día de hoy.

<!-- <div style="text-align:center">
    <img src="./imgs/LLMs_ecosystem.png" width="600">
</div> -->

<div style="text-align:center">
    <img src="https://lh3.googleusercontent.com/d/1sDIJQNmbtU7QHG6TkfA728QubnEmTx0w" width="600">
</div>



Como comentamos en el *fastbook*, en general, cuánto más grande es un modelo, más conocimiento y capacidad de "razonar", por eso ha habido tal explosión en el tamaño de estos:

<!-- <div style="text-align:center">
    <img src="./imgs/model_size_vol.png" width="700">
</div> -->

<div style="text-align:center">
    <img src="https://lh3.googleusercontent.com/d/1dNKHsHm9lsQsMNvqZmjUxVuqcN7YEDsj" width="700">
</div>

Para asegurar que todos los alumnos puedan correr los modelos y no tengan problemas con configurar el entorno, hardware, etc; vamos a usar un Google Colab como éste.
El inconveniente es que el *tier* gratuito de Google sólo incluye una máquina con 12 GB de RAM y una GPU con 15 GB de VRAM, y que no permite instalar ciertas librerías que optimizarían la ejecucción de modelos más grandes. Por ello, en esta sesión práctica **usaremos modelos más pequeños, cuya calidad no es equiparable a los modelos que usaríamos en un sistema en producción, pero que nos servirán igualmente para ilustrar una serie de conceptos**.



El modelo que usaremos hoy es de la familia [Qwen 2.5](https://qwenlm.github.io/blog/qwen2.5-llm/)

## <a id='toc1_'></a> Primeros pasos con un modelo de lenguaje

Recapitulamos: los modelos de lenguaje utilizan *tokenizers* ([link al visualizador](https://tiktokenizer.vercel.app/))

In [4]:
from transformers import AutoTokenizer
INSTRUCT_MODEL_CON_CASTELLANO = "Qwen/Qwen2.5-0.5B-Instruct" # ~2.4GB en la GPU
tokenizer = AutoTokenizer.from_pretrained(INSTRUCT_MODEL_CON_CASTELLANO)

# Veamos como tokeniza el textp

prompt = f"Hola modelito, hablas español?" # En general, preguntar a un LLM sobre sí mismo nunca da respuestas fiables, pero nos sirve para este ejemplo
prompt_tok = tokenizer.encode(prompt, return_tensors="pt") # shape: (1, n_tokens)
print(f"Traducción de la secuencia '{prompt}' a tokens:")
toks_as_list = prompt_tok[0].numpy().tolist()
print(toks_as_list)

Traducción de la secuencia 'Hola modelito, hablas español?' a tokens:
[68012, 1614, 6357, 11, 6055, 14493, 69888, 30]


In [5]:
vocab = tokenizer.get_vocab() # {txt: int}
inv_vocab = {v: k for k, v in vocab.items()} # {int: txt}
for tok_idx in toks_as_list:
    print(f"{tok_idx} -> {inv_vocab[tok_idx]}")

68012 -> Hola
1614 -> Ġmodel
6357 -> ito
11 -> ,
6055 -> Ġhab
14493 -> las
69888 -> ĠespaÃ±ol
30 -> ?


Nota: el tokenizador de los modelos Qwen (al menos hasta noviembre de 2024) se basa en el tokenizador basado en BPE (byte-pair-encoding) de OpenAI, que es *open-source*. Aunque con algunas modificaciones para que funcione mejor en Chino y otras lenguas. Mas info sobre el tokenizador en https://arxiv.org/pdf/2309.16609  

In [6]:
DEVICE = "cuda:0"
model = AutoModelForCausalLM.from_pretrained(INSTRUCT_MODEL_CON_CASTELLANO).to(DEVICE)

Inspeccionemos algunas características del modelo

In [7]:
describir_modelo(model)

 - El modelo tiene 494.0 millones parámetros
 - Arquitectura: Qwen2ForCausalLM
 - Contexto máximo: 32,768 tokens
 - Formato de los parámetros: torch.bfloat16
 - Tamaño del vocabulario: 151,936 tokens


Cada modelo *instruction-tuned* tiene su propia plantilla de *prompting*, que hay que seguir

In [8]:
msj = f"Hola modelito, hablas español?" # En general, preguntar a un LLM sobre sí mismo nunca da respuestas fiables, pero nos sirve para este ejemplo
prompt = crear_prompt_chat_model(msj, tokenizer)
print(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Hola modelito, hablas español?<|im_end|>
<|im_start|>assistant



Generemos ahora nuestro primer mensaje

In [9]:
prompt = crear_prompt_chat_model("Hola modelito, hablas español?", tokenizer)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
gen_kwargs = {"max_new_tokens": 50}
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, device=DEVICE)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sí, estoy aquí para ayudarte. ¿En qué puedo ayudarte hoy?


## <a id='toc12_'></a> Ejemplo práctico: no generativos vs NO instruction tuned vs instruction tuned

**No generativos**

Puedes probar [este modelo de HuggingFace](https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student) que es un clasificador de sentimiento entrenado en varios idiomas incluido el castellano. Por ejemplo, prueba con "en esa fiesta había un ambiente muy turbio"

**Generativos NO instruction-tuned**

In [10]:
NO_INSTRUCT_MODEL_CON_CASTELLANO = "Qwen/Qwen2.5-0.5B"
no_instruct_model = AutoModelForCausalLM.from_pretrained(NO_INSTRUCT_MODEL_CON_CASTELLANO).to(DEVICE)
no_instruct_tokenizer = AutoTokenizer.from_pretrained(NO_INSTRUCT_MODEL_CON_CASTELLANO)

In [11]:
msj = "Hola modelito, hablas español?" # En general, preguntar a un LLM sobre sí mismo nunca da 
# respuestas fiables, pero nos sirve para este ejemplo
response, model_output = responde_al_mensaje(
    no_instruct_model, 
    no_instruct_tokenizer, 
    msj, 
    streamer=streamer, 
    gen_kwargs=gen_kwargs, 
    device=DEVICE
)

 ¿Cómo te llamas? ¿Qué haces aquí? ¿Qué haces aquí? ¿Qué haces aquí? ¿Qué haces aquí? ¿Qué haces aquí? ¿Qué haces aquí? ¿Qué haces aquí? ¿Qué


In [12]:
msj = "Caminando por la calle me encontré a"
response, model_output = responde_al_mensaje(
    no_instruct_model, 
    no_instruct_tokenizer, 
    msj, 
    streamer=streamer, 
    gen_kwargs=gen_kwargs, 
    device=DEVICE
)

 un hombre que me miraba con una expresión de desagrado. Me pregunté si era un hombre de negocios o un hombre de calle. Me pregunté si era un hombre de negocios o un hombre de calle. Me


**Generativos instruction-tuned**

In [13]:
INSTRUCT_MODEL_CON_CASTELLANO = "Qwen/Qwen2.5-0.5B-Instruct"
instruct_model = AutoModelForCausalLM.from_pretrained(INSTRUCT_MODEL_CON_CASTELLANO).to(DEVICE)
instruct_tokenizer = AutoTokenizer.from_pretrained(INSTRUCT_MODEL_CON_CASTELLANO)

In [14]:
msj = "Hola modelito, hablas español?" # En general, preguntar a un LLM sobre sí mismo nunca da 
# respuestas fiables, pero nos sirve para este ejemplo
response, model_output = responde_al_mensaje(
    instruct_model, 
    instruct_tokenizer, 
    crear_prompt_chat_model(msj, instruct_tokenizer),
    streamer=streamer, 
    gen_kwargs=gen_kwargs, 
    device=DEVICE
)

Sí, estoy aquí para ayudarte en español. ¿En qué puedo asistirte hoy?


In [15]:
msj = "Completa esta frase: Caminando por la calle me encontré a"
response, model_output = responde_al_mensaje(
    instruct_model, 
    instruct_tokenizer, 
    crear_prompt_chat_model(msj, instruct_tokenizer), 
    streamer=streamer, 
    gen_kwargs=gen_kwargs, 
    device=DEVICE
)

Caminando por la calle me encontré a una amiga en un parque.


In [16]:
msj = f"""
En el mensaje a continuación, identifica si el sentido es POSITIVO, NEGATIVO o NEUTRAL. 
Responde sólo con una de estas tres palabras: POSITIVO; NEGATIVO; NEUTRAL; 
Mensaje: En esa fiesta había un ambiente muy turbio
"""
response, model_output = responde_al_mensaje(
    instruct_model, 
    instruct_tokenizer, 
    crear_prompt_chat_model(msj, instruct_tokenizer), 
    streamer=streamer, 
    gen_kwargs=gen_kwargs, 
    device=DEVICE
)

NEGATIVO


## <a id='toc2_'></a>Estrategias y parámetros de generación

Cómo funciona esta generación a más bajo nivel? La mayoría de modelos generativos utilizan por debajo
un método llamado *multinomial sampling*

<!-- <div style="text-align:center">
    <img src="./imgs/next_token_prob.png" width="700">
</div> -->

<div style="text-align:center">
    <img src="https://lh3.googleusercontent.com/d/1M6q2UMXEYLsO1PbESozlMFb5xtm-_yfR" width="700">
</div>

[Link a visualizador](https://poloclub.github.io/transformer-explainer/)

In [17]:
# Veamos si este en concreto hace multinomial sample por defecto
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

Vamos a ver como podemos inspeccionar las probabilidades de cada token en cada paso

In [18]:
gen_kwargs = {'max_new_tokens': 50}

In [19]:
prompt = crear_prompt_chat_model("Hola modelito, hablas español?", tokenizer)
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)

Sí, estoy aquí para ayudarte. ¿Cómo puedo asistirte hoy?


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In [20]:
inspeccionar_probabilidades(model_output, response, tokenizer, top_n_probables=10)

Probabilidades asociadas a cada token - [Candidatos más probables]
  'S'(100%)- ['S'(100%)]
  'í'(100%)- ['í'(100%)]
  ','(100%)- [','(100%)]
  ' estoy'(88%)- [' estoy'(88%)' hab'(12%)]
  ' aquí'(72%)- [' aquí'(72%)' hab'(28%)]
  ' para'(100%)- [' para'(100%)]
  ' ayud'(100%)- [' ayud'(100%)]
  'arte'(100%)- ['arte'(100%)]
  '.'(38%)- [' en'(62%)'.'(38%)]
  ' ¿'(100%)- [' ¿'(100%)]
  'Cómo'(38%)- ['En'(62%)'Cómo'(38%)]
  ' puedo'(100%)- [' puedo'(100%)]
  ' as'(100%)- [' as'(100%)]
  'ist'(100%)- ['ist'(100%)]
  'ir'(100%)- ['ir'(100%)]
  'te'(100%)- ['te'(100%)]
  ' hoy'(100%)- [' hoy'(100%)]
  '?'(100%)- ['?'(100%)]
  '<|im_end|>'(100%)- ['<|im_end|>'(100%)]

In [21]:
prompt = crear_prompt_chat_model("¿Cuál es el mejor mes del año? Responde con una sóla palabra!", tokenizer)
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)
inspeccionar_probabilidades(model_output, response, tokenizer, top_n_probables=10)

Julio
Probabilidades asociadas a cada token - [Candidatos más probables]
  'Jul'(5%)- ['J'(48%)'Mar'(34%)'E'(7%)'ener'(6%)'Jul'(5%)]
  'io'(0%)- ['uli'(50%)'ul'(50%)]
  '<|im_end|>'(0%)- ['io'(100%)]

In [22]:
prompt = crear_prompt_chat_model("Completa la frase: caminando por la calle me encontré", tokenizer)
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)
inspeccionar_probabilidades(model_output, response, tokenizer, top_n_probables=10)

caminando por la calle me encontré con un amigo en una cafetería.
Probabilidades asociadas a cada token - [Candidatos más probables]
  'c'(0%)- ['cam'(100%)]
  'amin'(0%)- ['in'(100%)]
  'ando'(100%)- ['ando'(100%)]
  ' por'(100%)- [' por'(100%)]
  ' la'(100%)- [' la'(100%)]
  ' calle'(100%)- [' calle'(100%)]
  ' me'(100%)- [' me'(100%)]
  ' encontr'(100%)- [' encontr'(100%)]
  'é'(100%)- ['é'(100%)]
  ' con'(78%)- [' con'(78%)' en'(22%)]
  ' un'(60%)- [' un'(60%)' una'(40%)]
  ' amigo'(100%)- [' amigo'(100%)]
  ' en'(39%)- [' en'(39%)'.'(30%)' que'(15%)' y'(8%)'.

'(8%)]
  ' una'(48%)- [' el'(52%)' una'(48%)]
  ' caf'(32%)- [' caf'(32%)' ti'(31%)' de'(16%)' es'(8%)' par'(7%)' of'(6%)]
  'eter'(100%)- ['eter'(100%)]
  'ía'(100%)- ['ía'(100%)]
  '.'(83%)- ['.'(83%)'.

'(17%)]
  '<|im_end|>'(100%)- ['<|im_end|>'(100%)]

Ahora hemos incluido la probabilidades en el output con fines didácticos. Pero hay algunos casos en que podemos usarlas como parte de nuestro algoritmo.

Por cierto! el mensaje "de sistema", dependiendo del modelo, tiene más o menos impacto. Con el modelo que estamos usando, no tiene impacto

In [23]:
msj = f"Hola modelito, hablas español?" # En general, preguntar a un LLM sobre sí mismo nunca da respuestas fiables, pero nos sirve para este ejemplo
_ = responde_al_mensaje(
    model,
    tokenizer,
    crear_prompt_chat_model(msj, tokenizer, mensaje_sistema="jajksashajksdah"),
    streamer=streamer,
    gen_kwargs=gen_kwargs,
    device=DEVICE
    
)
print("--"*30)
_ = responde_al_mensaje(
    model,
    tokenizer,
    crear_prompt_chat_model(msj, tokenizer, mensaje_sistema="Always answer with the word: 'banana'"),
    streamer=streamer,
    gen_kwargs=gen_kwargs,
    device=DEVICE

)
print("--"*30)
_ = responde_al_mensaje(
    model,
    tokenizer,
    crear_prompt_chat_model(msj, tokenizer, mensaje_sistema="You are an unhelpful assistant that gives stupid answers"),
    streamer=streamer,
    gen_kwargs=gen_kwargs,
    device=DEVICE
)

Sí, estoy hablando en español. ¿En qué puedo ayudarte hoy?
------------------------------------------------------------
Sí, hago lo que me pides. ¿En qué puedo ayudarte hoy?
------------------------------------------------------------
Sí, estoy aquí para ayudarte. ¿Cómo puedo asistirte hoy?


Veamos ahora algunos parámetros útiles

In [24]:
# Temperatura - Probemos con diferentes valores
gen_kwargs = {
    "temperature": 0.7,
    "max_new_tokens": 50
}
prompt = crear_prompt_chat_model("Hola modelito, hablas español?", tokenizer)
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)
print("\n")
inspeccionar_probabilidades(model_output, response, tokenizer, top_n_probables=10)

Sí, estoy aquí para ayudarte en español. ¿Cómo puedo asistirte hoy?


Probabilidades asociadas a cada token - [Candidatos más probables]
  'S'(100%)- ['S'(100%)]
  'í'(100%)- ['í'(100%)]
  ','(100%)- [','(100%)]
  ' estoy'(88%)- [' estoy'(88%)' hab'(12%)]
  ' aquí'(72%)- [' aquí'(72%)' hab'(28%)]
  ' para'(100%)- [' para'(100%)]
  ' ayud'(100%)- [' ayud'(100%)]
  'arte'(100%)- ['arte'(100%)]
  ' en'(62%)- [' en'(62%)'.'(38%)]
  ' español'(100%)- [' español'(100%)]
  '.'(100%)- ['.'(100%)]
  ' ¿'(100%)- [' ¿'(100%)]
  'Cómo'(22%)- ['En'(78%)'Cómo'(22%)]
  ' puedo'(100%)- [' puedo'(100%)]
  ' as'(100%)- [' as'(100%)]
  'ist'(100%)- ['ist'(100%)]
  'ir'(100%)- ['ir'(100%)]
  'te'(100%)- ['te'(100%)]
  ' hoy'(100%)- [' hoy'(100%)]
  '?'(100%)- ['?'(100%)]
  '<|im_end|>'(100%)- ['<|im_end|>'(100%)]

In [25]:
prompt = crear_prompt_chat_model("Completa la frase: caminando por la calle me encontré", tokenizer)
gen_kwargs = {
    "temperature": 1.5,
    "max_new_tokens": 50
}
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)
print("\n")
inspeccionar_probabilidades(model_output, response, tokenizer, top_n_probables=10)

caminando por la calle me encontré una casa. 

La frase original es una traducción indirecta de inglés en español:

"Camino por la calle y encontré una casa."

La respuesta completa de palabras y frases se ve


Probabilidades asociadas a cada token - [Candidatos más probables]
  'c'(5%)- ['cam'(56%)'¡'(11%)'¿'(8%)'"'(8%)'c'(5%)'en'(5%)'Cam'(3%)'con'(3%)]
  'amin'(0%)- ['in'(90%)'ino'(10%)]
  'ando'(100%)- ['ando'(100%)]
  ' por'(100%)- [' por'(100%)]
  ' la'(100%)- [' la'(100%)]
  ' calle'(100%)- [' calle'(100%)]
  ' me'(100%)- [' me'(100%)]
  ' encontr'(100%)- [' encontr'(100%)]
  'é'(100%)- ['é'(100%)]
  ' una'(5%)- [' con'(42%)' en'(23%)' un'(7%)' al'(7%)' una'(5%)' __'(5%)' a'(5%)' ac'(3%)' sent'(3%)]
  ' casa'(11%)- [' gran'(15%)' casa'(11%)' her'(9%)' señ'(7%)' persona'(6%)' ti'(6%)' nueva'(6%)' ciudad'(5%)' mujer'(5%)' mult'(5%)]
  '.'(15%)- ['.'(15%)' en'(14%)' nueva'(14%)' de'(10%)'.

'(8%)','(8%)' abandon'(7%)' con'(6%)' vac'(5%)' bl'(5%)]
  ' 

'(8%)- ['<|im_end|>'(81%)' 

'

Nota: una temperatura alta en modelos más potentes no tiene por qué resultar en respuestas de baja calidad, pero sí en respuestas más creativas y menos predecibles.

In [26]:
# Top-k sampling
gen_kwargs = {
    "top_k": 2,
    "max_new_tokens": 50,
}
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)
inspeccionar_probabilidades(model_output, response, tokenizer, top_n_probables=5)

caminando por la calle me encontré con un amigo en una tienda de libros.
Probabilidades asociadas a cada token - [Candidatos más probables]
  'c'(0%)- ['cam'(100%)]
  'amin'(0%)- ['in'(100%)]
  'ando'(100%)- ['ando'(100%)]
  ' por'(100%)- [' por'(100%)]
  ' la'(100%)- [' la'(100%)]
  ' calle'(100%)- [' calle'(100%)]
  ' me'(100%)- [' me'(100%)]
  ' encontr'(100%)- [' encontr'(100%)]
  'é'(100%)- ['é'(100%)]
  ' con'(78%)- [' con'(78%)' en'(22%)]
  ' un'(60%)- [' un'(60%)' una'(40%)]
  ' amigo'(100%)- [' amigo'(100%)]
  ' en'(57%)- [' en'(57%)'.'(43%)]
  ' una'(48%)- [' el'(52%)' una'(48%)]
  ' ti'(49%)- [' caf'(51%)' ti'(49%)]
  'enda'(100%)- ['enda'(100%)]
  ' de'(100%)- [' de'(100%)]
  ' libros'(40%)- [' '(60%)' libros'(40%)]
  '.'(100%)- ['.'(100%)]
  '<|im_end|>'(100%)- ['<|im_end|>'(100%)]

In [27]:
# Num return sequences
gen_kwargs = {
    "num_return_sequences": 3,
    "temperature": 0.7,
    "max_new_tokens": 50,
}
prompt = crear_prompt_chat_model("Completa la frase: caminando por la calle me encontré", tokenizer)
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=None, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)
print("\n-----\n".join(response))

caminando por la calle me encontré en mi casa.
-----
caminando por la calle me encontré con un amigo en una cafetería.

Esta es una traducción simple y gramaticalmente correcta de la oración original. La frase se compone de dos partes:

1. "Camin
-----
caminando por la calle me encontré con una amiga en un restaurante local, disfrutando de una cena cómica y culinaria que había preparado ella mismo.


Hay otros muchos parámetros que influyen en la generación y que pueden ser útiles en un momento dado (ej. top_p, repetition_penalty,  etc)
para un listado completo, podéis ver la documentación del método `generate` de la librería `transformers`:

Además, es importante aclarar que estos parámetros generalmente son comunes a todos los modelos, incluido
los modelos *closed-source*. Por ejemplo, en la documentación de la API de los modelos de OpenAI, se pueden encontrar
muchos de estos parámetros: [link]


**Greedy decoding**

Consiste en seleccionar siempre el token con mayor probabilidad, en lugar de muestrear de la distribución multinomial.



In [88]:
gen_kwargs = {
    "do_sample": False,
    "max_new_tokens": 200,
}
prompt = crear_prompt_chat_model("Cuéntame un chiste", tokenizer)
response, model_output = responde_al_mensaje(model, tokenizer, prompt, streamer=streamer, gen_kwargs=gen_kwargs, return_scores=False, device=DEVICE)

Claro, aquí tienes uno:

¿Por qué los gatos no usan Facebook?

Para buscar "gato" en la página de Facebook.

Espero que te guste! ¿Hay algo más?


Notése que el *greedy decoding* es equivalente a multinomial sampling con *top_k=1* ó con *temperature=0.001*!

**Beam decoding**

El *beam decoding* es una estrategia de generación que se usa para mejorar la calidad de las respuestas generadas por el modelo. En lugar de generar una única secuencia, el modelo genera varias secuencias candidatas y elige la "mejor" de ellas. La "mejor" de ellas en este caso se define como la secuencia con la mayor probabilidad conjunta de tokens (es decir, la que tiene el producto de sus probabilidades de tokens más alto).

<!-- <div style="text-align:center">
    <img src="./imgs/beam_decoding.png" width="700">
</div> -->

<div style="text-align:center">
    <img src="https://lh3.googleusercontent.com/d/1cGyQAMEn1LWR1RWbhnl3yqC8tYfqW_Lp" width="700">
</div>

In [89]:
gen_kwargs = {
    "max_new_tokens": 200,
    "num_beams": 5,
    "do_sample": False,
}
# No se puede usar streamer si num_beams != 1
response, model_output = responde_al_mensaje(model, tokenizer, prompt, gen_kwargs=gen_kwargs, return_scores=True, device=DEVICE)
print(response[0])

Claro, aquí tienes un chiste para ti:

¿Por qué los pájaros no usan Facebook?

Porque ya tienen Instagram.


Hay otros métodos de decoding más sofisticados. Por ejemplo, contrastive search (más detalles en este post de [huggingface](https://huggingface.co/blog/introducing-csearch))